# Molecular Similarity

## Objectives

- Generete molecular fingerprints for a given molecule.
- Evaluate structural similarity between molecules using different molecular fingerpints and similarity metrics.

Many useful documents/papers describe various aspects of molecular similarity, including molecular fingerprints and similarity measures.  Please read these if you need more details.

- Getting Started with the RDKit in Python<br>
(https://www.rdkit.org/docs/GettingStartedInPython.html#fingerprinting-and-molecular-similarity)

- Fingerprint Generation, GraphSim Toolkit 2.4.2<br>
(https://docs.eyesopen.com/toolkits/python/graphsimtk/fingerprint.html)

- Chemical Fingerprints<br>
(https://docs.chemaxon.com/display/docs/Chemical+Fingerprints)

- Extended-Connectivity Fingerprints<br>
(https://doi.org/10.1021/ci100050t)



## 1. Fingerprint Generation

In [1]:
from rdkit import Chem

In [2]:
mol = Chem.MolFromSmiles('CC(C)C1=C(C(=C(N1CC[C@H](C[C@H](CC(=O)O)O)O)C2=CC=C(C=C2)F)C3=CC=CC=C3)C(=O)NC4=CC=CC=C4')

### 1-(1) MACCS keys

The MACCS key is a binary fingerprint (a string of 0's and 1's).  Each bit position represents the presence (=1) or absence (=0) of a pre-defined structural feature.  The feature definitions for the MACCS keys are available at:<br> https://github.com/rdkit/rdkit/blob/master/rdkit/Chem/MACCSkeys.py

In [3]:
from rdkit.Chem import MACCSkeys
fingerprint = MACCSkeys.GenMACCSKeys(mol)

In [4]:
print(type(fingerprint))

for i in range(len(fingerprint)):
    print(fingerprint[i], end='')

fingerprint.ToBitString()    # Alternative, easier way to convert it to a bitstring.



In [5]:
len(fingerprint)

Note that the MACCS key is **166-bit-long**, but RDKit generates a 167-bit-long fingerprint.  It is because the index of a list/vector in many programming languages (including python) begins at 0.  To use the original numbering of the MACCS keys (1-166) (rather than 0-165), the MACCS keys were implemented to be 167-bit-long, with Bit 0 being always zero. Because Bit 0 is set to OFF for all compounds, it does not affect the evaluation of molecular similarity.

These are some methods that allow you to get some additional information on the MACCS Keys.

In [6]:
print(fingerprint.GetNumBits())
print(fingerprint.GetNumOffBits())
print(fingerprint.GetNumOnBits())
print(fingerprint.ToBinary())

**Exercise 1a:** Generate the MACCS keys for the molecules represented by the following SMILES, and get the positions of the bits set to ON in each of the three fingerprints.  What fragments do these bit positions correspond to?  (the bit definitions are available at

In [7]:
smiles = [ 'C1=CC=CC=C1', # Benzene (Kekule)
           'c1ccccc1',    # Benzene ("Aromatized" carbons)
           'C1CCCCC1']     # Cyclohexene

In [8]:
# Write your code in this cell.




<br>**Write the fragment definition of the bits ON** (one is already provided for you as an example).

### 1-(2) Circular Fingerprints

Circular fingerprints are hashed fingerprints.  They are generated by exhaustively enumerating "circular" fragments (containing all atoms within a given radius from each heavy atom of the molecule) and then hashing these fragments into a fixed-length bitstring.  (Here, the "radius" from an atom is measured by the number of bonds that separates two atoms).

Examples of circular fingerprints are the extended-connectivity fingerprint (ECFPs) and their variant called FCFPs (Functional-Class Fingerprints), originally described in a paper by Rogers and Hahn (https://doi.org/10.1021/ci100050t).  The RDKit implementation of these fingerprints are called "Morgan Fingerprints" (https://www.rdkit.org/docs/GettingStartedInPython.html#morgan-fingerprints-circular-fingerprints).

In [9]:
from rdkit.Chem import AllChem
from rdkit.Chem import rdFingerprintGenerator

fingerprint_generator = rdFingerprintGenerator.GetMorganGenerator(radius=2, fpSize=1024)
fingerprint = fingerprint_generator.GetFingerprint(mol)

fingerprint.ToBitString()  

We can also display an image of the fragment that caused the bit to be equal to 1.

The default highlight colors for the Morgan bits indicate:

blue: the central atom in the environment
yellow: aromatic atoms
gray: aliphatic
ring atoms

In [16]:
# First let's draw the molecule
from rdkit.Chem import Draw
from rdkit.Chem.Draw import IPythonConsole
IPythonConsole.ipython_useSVG = True
IPythonConsole.drawOptions.addAtomIndices = False
IPythonConsole.drawOptions.addStereoAnnotation = False
mol = Chem.MolFromSmiles('c1ccccc1CC1CC1')
mol


In [21]:
# Now draw the fragment responsible for bit 872 being equal to 1

mol = Chem.MolFromSmiles('c1ccccc1CC1CC1')
fingerprint_generator = AllChem.GetMorganGenerator(radius=2)
additional_output = AllChem.AdditionalOutput()
additional_output.CollectBitInfoMap()
fingerprint = fingerprint_generator.GetFingerprint(mol, additionalOutput=additional_output)
bit_info = additional_output.GetBitInfoMap()

mfp2_svg = Draw.DrawMorganBit(mol, 872, bit_info, useSVG=True)
mfp2_svg


When comparing the rdkit's Morgan fingerprints with the ECFP/FCFP fingerprints, it is important to remember that the name of ECFP/FCFP fingerprints are suffixed with the **diameter** of the atom environments considered, while the Morgan Fingerprints take a **radius** parameter (e.g., the second argument "2" of GetMorganFingerprintAsBitVect() in the above code cell).  The Morgan fingerprint generated above (with a radius of 2) is comparable to the ECFP4 fingerprint (with a diameter of 4).

**Exercise 1b:** For the molecules below, generate the 512-bit-long Morgan Fingeprint comparable to the **FCFP6** fingerprint.

- Search for the compounds by name and get their SMILES strings.
- Generate the molecular fingerprints from the SMILES strings.
- Print the generated fingerprints.
- To generate FCFP (not ECFP), read the following document: https://www.rdkit.org/docs/GettingStartedInPython.html#morgan-fingerprints-circular-fingerprints

In [23]:
synonyms = [ 'diphenhydramine', 'cetirizine', 'fexofenadine', 'loratadine' ]


In [30]:
# Write your code in this cell


### 1-(3) Path-Based Fingeprints

Path-based fingerprints are also hashed fingerprints.  They are generated by enumerating linear fragments of a given length and hashing them into a fixed-length bitstring.  An example is the RDKit's topological fingeprint.  As described in the RDK documentation (https://www.rdkit.org/docs/GettingStartedInPython.html#topological-fingerprints), while this fingerprint can be generated using FingerprintMols.FingerprintMol(), it is recommended to use rdmolops.RDKFingerprint() to generate the fingerprint using non-default parameter values. 

In [32]:
from rdkit.Chem import rdmolops

mol = Chem.MolFromSmiles('C1=CC=CC=C1')
fingerprint = rdmolops.RDKFingerprint(mol, fpSize=2048, minPath=1, maxPath=7).ToBitString()
print(fingerprint)


mol = Chem.MolFromSmiles('CC(C)C1=C(C(=C(N1CC[C@H](C[C@H](CC(=O)O)O)O)C2=CC=C(C=C2)F)C3=CC=CC=C3)C(=O)NC4=CC=CC=C4')
fingerprint = rdmolops.RDKFingerprint(mol, fpSize=2048, minPath=1, maxPath=7).ToBitString()
print(fingerprint)
fingerprint = rdmolops.RDKFingerprint(mol, fpSize=2048, minPath=1, maxPath=6).ToBitString()
print(fingerprint)

### 1-(4) PubChem Fingerprint

The PubChem Fingerprint is a 881-bit-long binary fingerprint (ftp://ftp.ncbi.nlm.nih.gov/pubchem/specifications/pubchem_fingerprints.pdf).  Similar to the MACCS keys, it uses a pre-defined fragment dictionary.  The PubChem fingerprint for each compound in PubChem can be downloaded from PubChem.  However, because they are base64-encoded, they should be decoded into binary bitstrings or bitvectors.

Details about how to decode base64-encoded PubChem fingerprints is described on the last page of the PubChem Fingerprint specification (ftp://ftp.ncbi.nlm.nih.gov/pubchem/specifications/pubchem_fingerprints.pdf).  Below is a user-defined function that decodes a PubChem fingerprint into a bit string.

In [33]:
from base64 import b64decode

def PCFP_BitString(pcfp_base64) :

    pcfp_bitstring = "".join( ["{:08b}".format(x) for x in b64decode( pcfp_base64 )] )[32:913]
    return pcfp_bitstring


In [35]:
pcfps = 'AAADcYBgAAAAAAAAAAAAAAAAAAAAAAAAAAAwAAAAAAAAAAABAAAAGAAAAAAACACAEAAwAIAAAACAACBCAAACAAAgAAAIiAAAAIgIICKAERCAIAAggAAIiAcAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA=='

In [36]:
print( len(PCFP_BitString(pcfps)) )
print(PCFP_BitString(pcfps))

The generated bitstring can be converted to a bitvector that can be used for molecular similarity computation in RDKit (to be discussed in the next section).

In [37]:
from rdkit import DataStructs
bitvect = DataStructs.CreateFromBitString(PCFP_BitString(pcfps))
type(bitvect)

In [39]:
prolog = "https://pubchem.ncbi.nlm.nih.gov/rest/pug"

for name in synonyms:
    url = prolog + "/compound/name/" + name + "/property/Fingerprint2D/txt"
    res = requests.get(url)
    fingerprint = res.text.split()
    print(fingerprint)
  

## 2. Computation of similarity scores

In [40]:
import requests
import time

In [41]:
cids = [    54454,  # Simvastatin (Zocor)
            54687,  # Pravastatin (Pravachol)
            60823,  # Atorvastatin (Lipitor)
           446155,  # Fluvastatin (Lescol)   
           446157,  # Rosuvastatin (Crestor)
          5282452,  # Pitavastatin (Livalo)
         97938126 ] # Lovastatin (Altoprev)

Let's get the SMILES strings from PubChem, generate Mol objects from them, and draw their chemical structures.

In [42]:
prolog = "https://pubchem.ncbi.nlm.nih.gov/rest/pug"

str_cid = ",".join([ str(x) for x in cids])

url = prolog + "/compound/cid/" + str_cid + "/property/isomericsmiles/txt"
print(url)
res = requests.get(url)
smiles = res.text.split()
print(smiles)

In [43]:
from rdkit import Chem
from rdkit.Chem import Draw

mols = [ Chem.MolFromSmiles(x) for x in smiles ]
Chem.Draw.MolsToGridImage(mols, molsPerRow=4, subImgSize=(200,200), legends=[str(x) for x in cids] )

Now generate MACCS keys for each compound.

In [48]:
from rdkit import DataStructs
from rdkit.Chem import MACCSkeys

fingerprints = [ MACCSkeys.GenMACCSKeys(x) for x in mols ]
print(fingerprints)

Now let's compute the pair-wise similarity scores among them.  To make higher scores easier to find, they are indicated with the "\*" character(s).

In [52]:
for i in range(0, len(fingerprints)) :
    for j in range(i+1, len(fingerprints)) :
        score = DataStructs.FingerprintSimilarity(fingerprints[i], fingerprints[j])
        print(cids[i], "vs.", cids[j], ":", round(score, 3), end='')
        
        if ( score >= 0.85 ):
            print(" ****")
        elif ( score >= 0.75 ):
            print(" ***")
        elif ( score >= 0.65 ):
            print(" **")
        elif ( score >= 0.55 ):
            print(" *")
        else:
            print(" ")


By default, the similarity score is generated using the **Tanimoto** equation.  RDKit also supports other similarity metrics, including Dice, Cosine, Sokal, Russel, Kulczynski, McConnaughey, and Tversky.  The definition of these metrics is available at the LibreTexts page (https://bit.ly/2kx9NCd).

In [56]:
print("Tanimoto    :", round(DataStructs.TanimotoSimilarity(fingerprints[0], fingerprints[1]), 4))
print("Dice        :", round(DataStructs.DiceSimilarity(fingerprints[0], fingerprints[1]), 4))
print("Cosine      :", round(DataStructs.CosineSimilarity(fingerprints[0], fingerprints[1]), 4))
print("Sokal       :", round(DataStructs.SokalSimilarity(fingerprints[0], fingerprints[1]), 4))
print("McConnaughey:", round(DataStructs.McConnaugheySimilarity(fingerprints[0], fingerprints[1]), 4))

**Exercise 2a:**  Compute the Tanimoto similarity scores between the seven compounds used in this section, using the PubChem fingerprints

- Download the PubChem Fingerprint for the seven CIDs.
- Convert the downloaded fingerprints into bit vectors.
- Compute the pair-wise Tanimoto scores using the bit vectors.

In [26]:
# Write your code in this cell





## 3. Interpretation of similarity scores

Using molecular fingeprints. we can compute the similarity scores between molecules.  However, how should these scores be interpreted?  For example, the Tanimoto score between CID 60823 and CID 446155 is computed to be 0.662, but does it mean that the two compounds are similar?  How similar is similar?  The following analysis would help answer these questions.

**Step 1.** Randomly select 1,000 compounds from PubChem and download their SMILES strings.

In [57]:
import random
# random.seed(0)

cid_max = 138962044    # The maximum CID in PubChem as of September 2019

cids = []

for x in range(1000):
    cids.append(random.randint(1, cid_max + 1))

chunk_size = 100

if len(cids) % chunk_size == 0 :
    num_chunks = int( len(cids) / chunk_size )
else :
    num_chunks = int( len(cids) / chunk_size ) + 1

smiles = []
    
for i in range(num_chunks):

    if (i == 0):
        print("Processing chunk ", end='')
    
    print(i, end=' ')
    
    idx1 = chunk_size * i
    idx2 = chunk_size * (i + 1)
    str_cids = ",".join([ str(x) for x in cids[idx1:idx2]])

    url = prolog + "/compound/cid/" + str_cids + "/property/isomericsmiles/txt"
    res = requests.get(url)

    if ( res.status_code == 200) :
        smiles.extend( res.text.split() )
    else :
        print("Chunk", i, "Failed to get SMILES.")
        
    time.sleep(0.2)

print("Done!")
print("# Number of SMILES : ", len(smiles))


In [58]:
print(smiles[0])

**Step 2.** Generate the MACCSKeys for each compound.

In [61]:
mols = [ Chem.MolFromSmiles(x) for x in smiles if x != None ]
fingerprints  = [ MACCSkeys.GenMACCSKeys(x) for x in mols if x != None ]
print("# Number of compounds:", len(mols))
print("# Number of fingerprints:", len(fps))

**Step 3.** Compute the Tanimoto scores between compounds.

In [62]:
print("# The number of compound pairs:", (len(fingerprints) * (len(fingerprints) - 1))/2 )

In [63]:
scores = []

for i in range(0, len(fingerprints)) :

    if (i == 0) :
        print("Processing compound ", end='')
    
    if (i % 100 == 0) :
        print(i, end=' ')
    
    for j in range(i+1, len(fingerprints)) :
        scores.append(DataStructs.FingerprintSimilarity(fingerprints[i], fingerprints[j]))

print("Done!")
print("# Number of scores : ", len(scores))

**Step 4.** Generate a histogram that shows the distribution of the pair-wise scores.

In [64]:
import matplotlib.pyplot as plt
%matplotlib inline

In [66]:
mybins = [ x * 0.01 for x in range(101)]

fig = plt.figure(figsize=(8,4), dpi=300)

plt.subplot(1, 2, 1)
plt.title("Distribution")
plt.hist(scores, bins=mybins)

plt.subplot(1, 2, 2)
plt.title("Cumulative Distribution")
plt.hist(scores, bins=mybins, density=True, cumulative=1)
plt.plot([0,1],[0.95,0.95])


In [67]:
for i in range(21) :
    thresh = i / 20
    num_similar_pairs = len([x for x in scores if x >= thresh]) 
    prob = num_similar_pairs / len(scores) * 100
    print("%.3f %8d (%8.4f %%)" % (thresh, num_similar_pairs, round(prob,4)))
    

In [68]:
print("Average:", sum(scores)/len(scores))

From the distribution of the similarity scores among 1,000 compounds, we observe the following:<br>
- If you randomly select two compounds from PubChem, the similarity score between them (computed using the Tanimoto equation and MACCS keys) is ~0.35 on average.
- About %5 of randomly selected compound pairs have a similarity score greater than 0.55.
- About %1 of randomly selected compound pairs have a similarity score greater than 0.65.

If two compounds have a Tanimoto score of 0.35, it is close to the avaerage Tanimoto score between randomly selected compounds and there is a 50% chance that you will get a score of 0.35 or greater just by selecting two compounds from PubChem.  Therefore, it is reasonable to consider the two compounds are not similar.<br>

The Tanimoto index may have a value ranging from 0 (for no similarity) to 1 (for identical molecules) and the midpoint of this value range is 0.5.  Because of this, a Tanimoto score of **0.55** may not sound great enough to consider two compounds to be similar.  However, according to the score distribution curve generated here, only **~5%** of randomly selected compound pairs will have a score greater than this.<br>

In the previous section, we computed the similarity scores between some cholesterol-lowering drugs, and CID 60823 and CID 446155 had a Tanimoto score of **0.662**.  Based on the score distribution curve generated in the second section, we can say that the probablilty of two randomly selected compounds from PubChem having a Tanimoto score greater than 0.662 is **less than 1%**.

The following code cell demonstrates how to find an appropriate similarity score threshold above which a given percentage of the compound pairs will be considered to be similar to each other.

In [69]:
scores.sort()    # Sort the scores in an increasing order.

In [ ]:
# to find a threshold for top 3% compound pairs (i.e., 97% percentile)
print("# total compound pairs:   ", len(scores))
print("# 95% of compound pairs:  ", len(scores) * 0.97)
print("# score at 95% percentile:", scores[ round(len(scores) * 0.97) ] )


**Exercise 3a:** In this exercise, we want to generate the distribution of the similarity scores among 1,000 compounds randomly selected from PubChem, using different molecular fingeprints and similarity metrics.<br>
For molecular fingerprints, use the following:
- PubChem Fingerprint
- MACCS keys
- Morgan Fingerprint (ECFP4 analogue, 1024-bit-long)

For similarity metrics, use the following:
- Tanimoto similarity

Here are additional instructions to follow:
- When generating the histograms, bin the scores from 0 to 1 with an increment of 0.01.
- For each distribution curve, determine the similarity score threshold so that **1%** of the compound pairs have a similarity score greater than or equal to this threshold.
- Use RDKit to generate the MACCS keys and Morgan fingerprint and download the PubChem fingerprints from PubChem.
- For reproducibility, use **random.seed(2019)** before you generate random CIDs.

**Step 1:** Generate 1,000 random CIDs, download the isomeric SMILES for them, and create the RDKit mol objects from the downloaded SMILES strings.

In [38]:
# Write your code in this cell





**Step 2:**  Generate the fingerprints, compute the similarity scores, determine similarity thresholds, and make histograms.

In [39]:
# Write your code in this cell



